In [ ]:
mjolnir_path = "../../../mjolnir"

#base_dir = "rtc_gridstd_p1e7_iso_thomas"
#base_dir = "../thor-data/alf_w_rt500_fixes/"
#base_dir = "waspalf/"
#base_dir = "waspalf_nothomas_short/"
#base_dir = "waspalf_nothomas_long/"
#base_dir = "../../../../../../../storage/THOR/thor-data/experiments/ubelix/mclmn/wasp43b/fix/w43_r5_ni_c1em13_Tm1530_A1p4e6_P20e7_plkfx/"
base_dir = "../../../../thor-data/w43_r5_ni_c1em17_Tm1530_A1p4e6_P20e7/"

#base_dir = "../../../../thor-data/w43_ni_c001_asciidmp_noustr/"

In [ ]:
# path to mjolnir code
import sys
sys.path.append(mjolnir_path) 

import pathlib
import re

import h5py
import imageio
import IPython.display as disp

import math

#%matplotlib ipympl
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm, SymLogNorm, Normalize
import numpy as np
from PIL import Image, ImageDraw, ImageFont, ImageOps

from scipy.optimize import root_scalar
# from pygifsicle import optimize

import hamarr as ham
import mjolnir_plot_helper as mph
from file_helpers import get_path_matching_regex, get_path_matching_regex_with_groups

In [ ]:
from cycler import cycler


import pandas as pd

In [ ]:
custom_cycler = (cycler(linestyle=['-', '--', '-.', ':']) * cycler(color=plt.rcParams['axes.prop_cycle'].by_key()['color'])      )


# plt.rc('lines', linewidth=4)

# set a custom cycler
plt.rc('axes', prop_cycle=custom_cycler)


In [ ]:
alfdmp = pathlib.Path(f"{base_dir}/alfprof/")

In [ ]:
FIGSIZE_x = 15
FIGSIZE_y = 11
dpi = 100


FIGSIZE = (FIGSIZE_x, FIGSIZE_y)

In [ ]:
# use col_idx specified with mu_star plot, to help find correct column
# col_idx = 2384
step_idx = 156667
#step_idx = 144002
num_lay = 40
num_bins = 35
iso = False
has_hl = False

## read THOR Alf output file list

In [ ]:
def col2dct(regmatch):
    out = {}
    for d in regmatch:
        out[int(d['groups'][0])] = d['path']
    return out



steps = col2dct(get_path_matching_regex_with_groups(alfdmp, "step_(\d+)"))

In [ ]:
columns = col2dct(get_path_matching_regex_with_groups(steps[step_idx], "column_(\d+)"))

In [ ]:
def load_columns_data(columns, dataset):
    out = {}
    for k, p in columns.items():
        out[k] = p / dataset
    return out

In [ ]:
init_profs = load_columns_data(columns, "tpprofile_init.dat");
plk_profs = load_columns_data(columns, "plkprofile.dat");
plk_int_profs = load_columns_data(columns, "plkintprofile.dat");




In [ ]:




interp_profs = load_columns_data(columns, "tprofile_interp.dat")
opac_profs = load_columns_data(columns, "opacprofile.dat")
opac_int_profs = load_columns_data(columns, "opacintprofile.dat")


deltacolmass_profs = load_columns_data(columns, "deltacolmassprofile.dat")

deltacolupper_profs = load_columns_data(columns, "deltacolupperprofile.dat")
deltacollower_profs = load_columns_data(columns, "deltacollowerprofile.dat")
    
meanmolmass_profs = load_columns_data(columns, "meanmolmassprofile.dat")
meanmolmassint_profs = load_columns_data(columns, "meanmolmassintprofile.dat")

opt_depth_profs = load_columns_data(columns, "opt_depthprofile.dat")
opt_depth_upper_profs = load_columns_data(columns, "opt_depth_upper_profile.dat")
opt_depth_lower_profs = load_columns_data(columns, "opt_depth_lower_profile.dat")


singscat_band_profs = load_columns_data(columns, "single_scat_band_profile.dat")

Fu_profs = load_columns_data(columns, "F_up_profile.dat")
Fd_profs = load_columns_data(columns, "F_down_profile.dat")
Fdir_profs = load_columns_data(columns, "F_dir_profile.dat")
Fcdir_profs = load_columns_data(columns, "Fc_dir_profile.dat")

In [ ]:
trans_band_profs = load_columns_data(columns, "trans_band_profile.dat")

trans_band_upper_profs = load_columns_data(columns, "trans_band_upper_profile.dat")
trans_band_lower_profs = load_columns_data(columns, "trans_band_lower_profile.dat")

In [ ]:
singscat_band_upper_profs = load_columns_data(columns, "single_scat_band_upper_profile.dat")
singscat_band_lower_profs = load_columns_data(columns, "single_scat_band_lower_profile.dat")

In [ ]:

if iso:
    M_profs = load_columns_data(columns, "M_profile.dat")
    N_profs = load_columns_data(columns, "N_profile.dat")
    P_profs = load_columns_data(columns, "P_profile.dat")
    G_plus_profs = load_columns_data(columns, "G_plus_profile.dat")
    G_minus_profs = load_columns_data(columns, "G_minus_profile.dat")
    

    
if not iso:
    M_upper_profs = load_columns_data(columns, "M_upper_profile.dat")
    M_lower_profs = load_columns_data(columns, "M_lower_profile.dat")

    N_upper_profs = load_columns_data(columns, "N_upper_profile.dat")
    N_lower_profs = load_columns_data(columns, "N_lower_profile.dat")
    
    P_upper_profs = load_columns_data(columns, "P_upper_profile.dat")
    P_lower_profs = load_columns_data(columns, "P_lower_profile.dat")

    G_plus_upper_profs = load_columns_data(columns, "G_plus_upper_profile.dat")
    G_plus_lower_profs = load_columns_data(columns, "G_plus_lower_profile.dat")


    G_minus_upper_profs = load_columns_data(columns, "G_minus_upper_profile.dat")
    G_minus_lower_profs = load_columns_data(columns, "G_minus_lower_profile.dat")
    X_buff_profs = load_columns_data(columns, "X_thomas_profile.dat")


In [ ]:
interface_profs = load_columns_data(columns, "tpprofile_interface.dat")

In [ ]:
alf_qheat_profs = load_columns_data(columns, "alf_qheat_profile.dat")
dgrt_qheat_profs = load_columns_data(columns, "dgrt_qheat_profile.dat")

# Col Lat Lon and column selection

By playing with col_idx and lookoing at TP profile, we can find the interesting column here

In [ ]:

latlon = np.zeros((len(init_profs), 2))
mu_star_col = np.zeros((len(init_profs)))
ps_toa = np.zeros((len(init_profs)))
ps_boa = np.zeros((len(init_profs)))

for k, p in init_profs.items():
    f = p.open()
    line1 = f.readline()
    
    
    m = re.match("#.* lat: \[(.*)\] lon: \[(.*)\] mustar: \[(.*)\] P_BOA: \[(.*)\] P_TOA: \[(.*)\]", line1)
    if m is not None:
        # print(tp, ":", line1)
        # print("match: ", m.group(1), m.group(2), m.group(3))
        lat = float(m.group(1))
        lon = float(m.group(2))
        mustar = float(m.group(3))
        p_boa =  float(m.group(4))
        p_toa =  float(m.group(5))
    else:
        raise Exception("No match for config in TP file")
        
    if lat < 0.0:
        lat += 360.0
    latlon[k,:] = [lat, lon]
    mu_star_col[k] = mustar
    ps_toa[k] = p_toa
    ps_boa[k] = p_boa
    


In [ ]:
#col_idx = 5*16*16
#col_idx = 2540
col_idx = 848
col_idx_offset_neg = 0
col_idx_offset_pos = 0
col_marker = list(range(col_idx - col_idx_offset_neg,col_idx+col_idx_offset_pos,1))

In [ ]:

fig, ax = plt.subplots(1, 1, figsize=(1*FIGSIZE_x, FIGSIZE_y), dpi=dpi)

ax.scatter(latlon[:,0], latlon[:,1], c=mu_star_col)

for col in col_marker:
    ax.scatter(latlon[col,0], latlon[col,1], marker='x', s=0.5, linewidth=10)
    
ax.scatter(latlon[col_idx,0], latlon[col_idx,1], marker='+', s=1.5, linewidth=10)

ax.grid(True)

mu_star = mu_star_col[col_idx]

In [ ]:
n = col_idx
delta = 1
stride = 1



In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(1*FIGSIZE_x, FIGSIZE_y), dpi=dpi)
for i in range(n, n + delta, stride):
    init_table_col = pd.read_csv(init_profs[i], sep='\s+', header=None, skiprows=2, names=['T','P','rho'])
    
    
    
    ax.plot(  init_table_col['T'], init_table_col['P'], label=f'col[{i}]')


    
ax.set(xscale='linear', xlabel='Temperature')
ax.set(yscale='log', ylabel='pressure [bar]')
ax.invert_yaxis()
ax.grid(True)
#ax.legend()

# load helios data

In [ ]:
hldmp =  pathlib.Path("../HELIOS-orig/alfdbg/")

hl_subdir = hldmp / f"alfprof_disc_{col_idx}" 
hl_planet = "alfprof_disc"

In [ ]:
tpfile = hldmp / f"alfprof_{col_idx}" /  f"alfprof_{col_idx}_tp.dat" 

if has_hl:
    hl_tp =  pd.read_csv(tpfile, sep='\s+', header=0, skiprows=1) #, names = [
    # "layer",   "temp[K]",          "press.[10^-6bar]",        "altitude[cm]",         "height.of.layer[cm]",    "conv.unstable?[1:yes,0:no]",    "conv.lapse-rate?[1:yes,0:no]"
        #"layer",   "temp",          "press",        "altitude",         "height.of.layer",    "conv.unstable",    "conv.lapse-rate"
    #])

    hl_tp['press'] = hl_tp['press.[10^-6bar]']*1e-6
    hl_tp['temp'] = hl_tp['temp.[K]']

    # get rid of ghost layer
    hl_tp_lay = hl_tp.loc[1:, : ]
    hl_tp_lay

In [ ]:
# input values from thor
init_table = pd.read_csv(init_profs[col_idx], sep='\s+', header=None, skiprows=2, names=['T','P','rho'])
# layer values (same as input)
interp_table = pd.read_csv(interp_profs[col_idx], sep='\s+', header=None, skiprows=2, names=['T'])
# interpolated interface values
interface_table = pd.read_csv(interface_profs[col_idx], sep='\s+', header=None, skiprows=2, names=['T','P','rho'])

In [ ]:
interface_table

In [ ]:
fig, ((T_ax1, T_ax2), (P_ax1, TP_ax2), (rho_ax1, rho_ax2)) = plt.subplots(3, 2, figsize=(2*FIGSIZE_x, 3*FIGSIZE_y), dpi=dpi)


if has_hl:
    T_ax1.plot(hl_tp_lay['temp'].values, label="hl")    
    T_ax2.plot(hl_tp_lay['temp'], hl_tp_lay['press'], label='hel')
    
center_val = np.arange(len(init_table['T'].values)) + 0.5
center_val2 = np.arange(len(interp_table['T'].values)) - 0.5
interface_val = np.arange(len(interface_table['T'].values))

layer_pressure = init_table['P'].values
interface_pressure = interface_table['P'].values

T_ax1.plot(center_val, init_table['T'].values, label="init", linestyle='None', marker='o')
T_ax1.plot(center_val2, interp_table['T'].values, label="layer", linestyle='None', marker='x')
T_ax1.plot(interface_val, interface_table['T'].values, label="interface", linestyle='None', marker='+')
T_ax1.grid(True)
T_ax1.legend()

T_ax1.set(xlabel="Interface index",
            ylabel="Temperature [K]")

P_ax1.plot(center_val, init_table['P'].values, label="init", linestyle='None', marker='o')
P_ax1.plot(center_val, layer_pressure, label="layer", linestyle='None', marker='x')
P_ax1.plot(interface_val, interface_table['P'].values, label="interface", linestyle='None', marker='+')


P_ax1.legend()
#P_ax1.set_xlim(35,40)

P_ax1.set(yscale='log', ylabel="Pressure [Bar]",
       xscale='linear', xlabel="level")
P_ax1.invert_yaxis()


P_ax1.grid(True)

T_ax2.plot(init_table['T'], init_table['P'], label='Alf init')

T_ax2.set(yscale='log', ylabel="Pressure [Bar]",
       xscale='linear', xlabel="Temperature [K]")
T_ax2.invert_yaxis()

T_ax2.legend()
T_ax2.grid(True)
#ax2.plot()


TP_ax2.plot( init_table['T'].values, init_table['P'].values, label="init", linestyle='None', marker='o')
TP_ax2.plot( interp_table['T'].values, np.concatenate((np.array([layer_pressure[0] - 0.5*(layer_pressure[1]-layer_pressure[0])]), layer_pressure)), label="layer", linestyle='None', marker='x')
TP_ax2.plot( interface_table['T'].values, interface_table['P'].values, label="interface", linestyle='None', marker='+')
TP_ax2.grid(True)
TP_ax2.legend()

TP_ax2.set(xlabel="Temperature [K]",
            ylabel="Pressure [Bar]", yscale='log')

TP_ax2.invert_yaxis()

TP_ax2.legend()


rho_ax1.plot( center_val, init_table['rho'].values, label="layer", linestyle='None', marker='o')
#rho_ax1.plot( interp_table['T'].values, np.concatenate((np.array([layer_pressure[0] - 0.5*(layer_pressure[1]-layer_pressure[0])]), layer_pressure)), label="layer", linestyle='None', marker='x')
rho_ax1.plot( interface_val, interface_table['rho'].values, label="interface", linestyle='None', marker='+')
rho_ax1.grid(True)
rho_ax1.legend()

rho_ax1.set(xlabel="interface index",
            ylabel="density", yscale='log')

rho_ax1.invert_yaxis()

rho_ax1.legend()



offset above between helios and thor is because of different height grids. gets corrected in pressure diplay

# TP Profile

In [ ]:
col_idx

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(1*FIGSIZE_x, FIGSIZE_y), dpi=dpi)

n = 4*16*16
n=0

delta = 2562

stride = 1
rng = list(range(n, n + delta, stride)) #+ [col_idx]

vert_unit_pressure = True

for i in rng:
    init_table_col = pd.read_csv(init_profs[i], sep='\s+', header=None, skiprows=2, names=['T','P','rho'])
    
    if vert_unit_pressure:
        vert_unit = init_table_col['P']
    else:
        vert_unit = range(0, len(init_table_col['P']))
    
    ax.plot(  init_table_col['T'], vert_unit, label=f'col[{i}]')


    
ax.set(xscale='linear', xlabel='Temperature')

if vert_unit_pressure:
    ax.set(yscale='log', ylabel='pressure [bar]')
    ax.invert_yaxis()
else:
    ax.set(yscale='linear', ylabel='level')

ax.grid(True)
#ax.legend()

# plank function

In [ ]:
if has_hl:
    hlpkfile = hl_subdir /  f"{hl_planet}_{col_idx}_planck_cent.dat" 

    hl_pk =  pd.read_csv(hlpkfile, sep='\s+', header=0, skiprows=2)

th_plk_table = pd.read_csv(plk_profs[col_idx], sep='\s+', header=0)

if not iso:
    th_plk_int_table = pd.read_csv(plk_int_profs[col_idx], sep='\s+', header=0)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(2*FIGSIZE_x, FIGSIZE_y), dpi=dpi)


th_pk_table2 = th_plk_table.loc[:,'layer[0]':f'layer[{num_lay-1}]'].values
if not iso:
    th_plk_int_table2 = th_plk_int_table.loc[:,'int[0]':f'int[{num_lay}]'].values


c_cycler =  cycler(color=plt.rcParams['axes.prop_cycle'].by_key()['color'])()



for nbin in range(num_bins):
    cl = next(c_cycler)['color']
    init_table_col = pd.read_csv(init_profs[i], sep='\s+', header=None, skiprows=2, names=['T','P','rho'])
    ax1.plot(  th_pk_table2[nbin,:], layer_pressure, label='th', ls='None', marker='x', ms=3, color=cl)
    if not iso:
        ax1.plot(  th_plk_int_table2[nbin,:], interface_pressure, label='th', ls='None', marker='+', ms=2, color=cl)

if has_hl:
    hl_pk_table = hl_pk.loc[:,'B_lay[0]':f'B_lay[{num_lay-1}]'].values
    ax1.plot( hl_pk_table[nbin,:]*1e-1,hl_tp_lay['press'], label='hl')
ax1.grid(True)
ax1.set(xscale='linear', xlabel='plank')
ax1.set(yscale='log', ylabel='pressure')
ax1.invert_yaxis()

#ax1.legend()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(2*FIGSIZE_x, FIGSIZE_y), dpi=dpi)

if has_hl:
    hl_toa = hl_pk['Planck_T_star']*1e-1
    ax1.plot(hl_toa, label='hl')
    hl_boa = hl_pk['Planck_T_intern']*1e-1
    ax2.plot(hl_boa, label='hl')    
    
th_toa = th_plk_table['layer[TOA]']

ax1.plot(th_toa, label='th')

ax1.legend()




th_boa = th_plk_table['layer[BOA]']


ax2.plot(th_boa, label='th')
ax2.set(yscale='log')
ax2.legend()



# opacities

In [ ]:
th_opac_table = pd.read_csv(opac_profs[col_idx], sep='\s+', header=0)

In [ ]:
if has_hl:
    hlopacfile =  hl_subdir /  f"{hl_planet}_{col_idx}_opacities.dat" 

    hl_opac =  pd.read_csv(hlopacfile, sep='\s+', header=0, skiprows=2)

In [ ]:
th_opac_table;

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(2*FIGSIZE_x, FIGSIZE_y), dpi=dpi)


if has_hl:
    hl_oc_table = hl_opac.loc[:,'opac_lay[0]':f'opac_lay[{num_lay-1}]'].values
    
    for nbin in range(num_bins):
    #nbin = 34
        ax2.plot( hl_oc_table[nbin,:]*1e-1,hl_tp_lay['press'], label='hl')


th_opac_table = pd.read_csv(opac_profs[col_idx], sep='\s+', header=0)
if not iso:
    th_opac_int_table = pd.read_csv(opac_int_profs[col_idx], sep='\s+', header=0)

col_TP_table = pd.read_csv(init_profs[col_idx], sep='\s+', header=None, skiprows=2, names=['T','P','rho'])

th_oc_table = th_opac_table.loc[:,'layer[0]':f'layer[{num_lay-1}]'].values

if not iso:
    th_oc_int_table = th_opac_int_table.loc[:,'interface[0]':f'interface[{num_lay}]'].values

c_cycler =  cycler(color=plt.rcParams['axes.prop_cycle'].by_key()['color'])()

for nbin in range(num_bins):
#nbin = 34
    cl = next(c_cycler)['color']
    ax1.plot(  th_oc_table[nbin,:], col_TP_table['P'], label=f'th[{nbin}]', ls='None', marker='x', color = cl, ms=4)
    if not iso:
        ax1.plot(  th_oc_int_table[nbin,:], interface_pressure, label=f'th[{nbin}]', ls='None', marker='+', color = cl, ms=4)

ax1.set(xscale='log', xlabel='opacities')
ax1.set(yscale='log', ylabel='pressure')
ax1.invert_yaxis()

#ax1.legend()

ax2.set(xscale='linear', xlabel='opacities')
ax2.set(yscale='log', ylabel='pressure')
ax2.invert_yaxis()

ax2.legend()



# mean molecular weight

In [ ]:

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(2*FIGSIZE_x, FIGSIZE_y), dpi=dpi)

if has_hl:
    hlmmmfile = hl_subdir /  f"{hl_planet}_{col_idx}_colmass_mu_cp_kappa_entropy.dat" 

    hl_mmm =  pd.read_csv(hlmmmfile, sep='\s+', header=None, skiprows=2)

    hl_mmm_table = hl_mmm[3].values
    
    ax1.plot( hl_mmm_table,hl_tp_lay['press'], label='hl')

    
th_mmm_table = pd.read_csv(meanmolmass_profs[col_idx], sep='\s+', header=0)
th_mmm_table = th_mmm_table['meanmolmass'].values
#col_TP_table = pd.read_csv(, sep='\s+', header=None, skiprows=2, names=['T','P'])


ax1.plot(  th_mmm_table, layer_pressure, label='layer', ls=':', marker='x')
if not iso:
    th_mmm_interface_table = pd.read_csv(meanmolmassint_profs[col_idx], sep='\s+', header=0)['meanmolmassint'].values
    

    ax1.plot(  th_mmm_interface_table, interface_pressure, label='interface', linestyle="None", marker='x')

ax1.set(xscale='linear', xlabel='mean molecular mass')
ax1.set(yscale='log', ylabel='pressure')
ax1.invert_yaxis()

ax1.legend()

# delta col mass

In [ ]:

# reuse above file from MMM that contains also delta_col_mass

vert_unit_pressure = False

if iso:

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(2*FIGSIZE_x, FIGSIZE_y), dpi=dpi)

    
        
    for i in range(0,2562, 100):
        th_dcm_table = pd.read_csv(deltacolmass_profs[i], sep='\s+', header=0)
        th_dcm_table = th_dcm_table['delta_col_mass'].values
        
        col_TP_table = pd.read_csv(init_profs[i], sep='\s+', header=None, skiprows=2, names=['T','P','rho'])
    
        ax1.plot(  th_dcm_table, col_TP_table['P'], label=f'th_col[{i}]')
    
    if has_hl:
        hl_dcm_table = hl_mmm[2].values*10
        for i in range(0,2562, 100):
            ax2.plot( hl_dcm_table,hl_tp_lay['press'], label='hl')

    ax1.set(xscale='log', xlabel='delta col mass THOR [kg m^-2]')
    ax1.set(yscale='log', ylabel='pressure')
    ax1.invert_yaxis()
    
    ax1.legend()

    ax2.set(xscale='log', xlabel='delta col mass HELIOS [kg m^-2]')
    ax2.set(yscale='log', ylabel='pressure')
    ax2.invert_yaxis()

    ax2.legend()
else:
    fig, ((ax1_upper, ax2_upper), (ax1_lower, ax2_lower), (ax1_both, ax2_both))  = plt.subplots(3, 2, figsize=(2*FIGSIZE_x, 2*FIGSIZE_y), dpi=dpi)

    # get an instance of a color cycler
    c_cycler =  cycler(color=plt.rcParams['axes.prop_cycle'].by_key()['color'])()
    
    
    for i in range(848,849, 1):
        # get next color from color cycler iterator
        cl = next(c_cycler)['color']
        
        if vert_unit_pressure:
            col_TP_table = pd.read_csv(init_profs[i], sep='\s+', header=None, skiprows=2, names=['T','P','rho'])['P'].values
        else:
            col_TP_table = np.arange(len(layer_pressure))
        
        th_dcm_col_upper = pd.read_csv(deltacolupper_profs[i], sep='\s+', header=0)['delta_col_upper'].values
        

        
    
        ax1_upper.plot(  th_dcm_col_upper, col_TP_table, label=f'th_col_upper[{i}]')
        
        th_dcm_col_lower = pd.read_csv(deltacollower_profs[i], sep='\s+', header=0)['delta_col_lower'].values
        

        
    
        ax1_lower.plot(  th_dcm_col_lower, col_TP_table, label=f'th_col_lower[{i}]')
    
        ax1_both.plot(  th_dcm_col_upper, col_TP_table + 0.75, label=f'th_col_upper[{i}]', ls='None', marker=6, ms=5.5, color = cl)
        ax1_both.plot(  th_dcm_col_lower, col_TP_table + 0.25, label=f'th_col_lower[{i}]', ls='None', marker=7, ms=5.5, color = cl )
        ax2_both.plot(  th_dcm_col_upper+th_dcm_col_lower, col_TP_table, label=f'th_both[{i}]', marker=6, ms=5.5, color = cl)
    if has_hl:
        hl_dcm_table = hl_mmm[2].values*10
        for i in range(0,2562, 100):
            ax2_upper.plot( hl_dcm_table,hl_tp_lay['press'], label='hl')

    if vert_unit_pressure:
        vunit = 'log'
    else:
        vunit = 'linear'
        
    ax1_upper.set(xscale='linear', xlabel='delta col mass upper [kg m^-2]')
    ax1_upper.set(yscale=vunit, ylabel='pressure')
    if vert_unit_pressure:
        ax1_upper.invert_yaxis()
    
    ax1_upper.legend()

    ax1_lower.set(xscale='linear', xlabel='delta col mass lower [kg m^-2]')
    ax1_lower.set(yscale=vunit, ylabel='pressure')
    if vert_unit_pressure:
        ax1_lower.invert_yaxis()
    
    ax1_lower.legend()
    
    ax2_upper.set(xscale='log', xlabel='delta col mass HELIOS [kg m^-2]')
    ax2_upper.set(yscale='log', ylabel='pressure')
    ax2_upper.invert_yaxis()

    ax2_upper.legend()
    

    ax1_both.set(xscale='log', xlabel='delta col mass THOR [kg m^-2]')
    ax1_both.set(yscale=vunit, ylabel='pressure')
    if vert_unit_pressure:
        ax1_both.invert_yaxis()
    #ax1_both.set_ylim((35,40))
   # ax1_both.legend()
    ax1_both.grid(True, axis='y')
    
    ax2_both.set(xscale='log', xlabel='delta col mass both [kg m^-2]')
    ax2_both.set(yscale=vunit, ylabel='pressure')
    if vert_unit_pressure:
        ax2_both.invert_yaxis()
    
    ax2_both.grid(True, axis='y')


# optical depth - delta tau 

In [ ]:



fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(2*FIGSIZE_x, 2*FIGSIZE_y), dpi=dpi)

if iso:
    nbin = 20
    for i in range(0,2562, 400):
        th_optd_table = pd.read_csv(opt_depth_profs[i], sep='\s+', header=0)


        th_optd_table = th_optd_table.loc[:,'layer[0]':f'layer[{num_lay-1}]'].values
    
        col_TP_table = pd.read_csv(init_profs[i], sep='\s+', header=None, skiprows=2, names=['T','P','rho'])
    
        ax1.plot(  th_optd_table[nbin,:], col_TP_table['P'], label='th',ls='None', marker='+', ms = 4)
        ax2.plot(  np.exp(-0.5*th_optd_table[nbin,:]), init_table['P'], label='th')

    ax1.set(xscale='linear', xlabel='optical_depth')
    ax1.set(yscale='log', ylabel='pressure')
    ax1.invert_yaxis()
    ax1.grid(True)
    #ax1.legend()

    
    ax2.set(xscale='linear', xlabel='exp(-E/epsilon*Delta_tau)')
    ax2.set(yscale='log', ylabel='pressure')
    ax2.invert_yaxis()

    #ax2.legend()
else:
    nbin = 20
    c_cycler =  cycler(color=plt.rcParams['axes.prop_cycle'].by_key()['color'])()
    
    
    
    
    
    for i in range(0,2562, 400):
        init_table_col = pd.read_csv(init_profs[i], sep='\s+', header=None, skiprows=2, names=['T','P','rho'])
        interface_table_col = pd.read_csv(interface_profs[i], sep='\s+', header=None, skiprows=2, names=['T','P','rho'])
        interface_pressure_col = interface_table_col['P'].values
        layer_pressure_col = init_table_col['P'].values
        
        cl = next(c_cycler)['color']
        
        th_optd_upper_table = pd.read_csv(opt_depth_upper_profs[i], sep='\s+', header=0)
        th_optd_upper_table = th_optd_upper_table.loc[:,'layer[0]':f'layer[{num_lay-1}]'].values
        th_optd_lower_table = pd.read_csv(opt_depth_lower_profs[i], sep='\s+', header=0)
        th_optd_lower_table = th_optd_lower_table.loc[:,'layer[0]':f'layer[{num_lay-1}]'].values
    
        press_upper = (interface_pressure_col[1:] + layer_pressure_col)/2.0
        press_lower = (interface_pressure_col[:-1] + layer_pressure_col)/2.0
        press_mean = (press_upper + press_lower)/2.0
        # optical depth for layer
        optd_lay = th_optd_upper_table + th_optd_lower_table
        
        ax1.plot(  th_optd_upper_table[nbin,:], press_upper, label='up',ls='None', marker=6, ms = 4, color=cl)
        ax2.plot(  np.exp(-0.5*th_optd_upper_table[nbin,:]), press_upper, label='th',ls='None', marker=6, ms = 4, color=cl)
        
        ax1.plot(  th_optd_lower_table[nbin,:], press_lower, label='dn',ls='None', marker=7, ms = 4, color=cl)
        ax2.plot(  np.exp(-0.5*th_optd_lower_table[nbin,:]), press_lower, label='th',ls='None', marker=7, ms = 4, color=cl)

        ax3.plot(  optd_lay[nbin,:], press_mean, label='mean',ls='None', marker='+', ms = 3, color=cl)
        
    ax1.set(xscale='log', xlabel='optical_depth')
    ax1.set(yscale='log', ylabel='pressure')
    ax1.invert_yaxis()
    ax1.grid(True)
    ax1.set_title("Upper and lower optical depth")
    #ax1.legend()

    
    ax2.set(xscale='linear', xlabel='exp(-E/epsilon*Delta_tau)')
    ax2.set(yscale='log', ylabel='pressure')
    ax2.invert_yaxis()

    #ax2.legend()
    
    ax3.set(xscale='log', xlabel='optical_depth')
    ax3.set(yscale='log', ylabel='pressure')
    ax3.invert_yaxis()
    ax3.set_title("Full layer optical depth")
    ax3.grid(True)

# transmission 

In [ ]:
if iso:
    fig, ax = plt.subplots(1, 1, figsize=(1*FIGSIZE_x, FIGSIZE_y), dpi=dpi)

    
    th_trans_table = pd.read_csv(trans_band_profs[col_idx], sep='\s+', header=0)

    th_trans_table = th_trans_table.loc[:,'layer[0]':f'layer[{num_lay-1}]'].values

    col_TP_table = pd.read_csv(init_profs[col_idx], sep='\s+', header=None, skiprows=2, names=['T','P','rho'])
    
    
    for nbin in range(35):
        ax.plot(  th_trans_table[nbin,:], col_TP_table['P'], label=f'th_bin[{nbin}]')

    ax.set(xscale='linear', xlabel='transmission')
    ax.set(yscale='log', ylabel='pressure')
    ax.invert_yaxis()

    #ax.legend()

In [ ]:
if not iso:
    fig, (ax_up, ax_lo) = plt.subplots(1, 2, figsize=(2*FIGSIZE_x, FIGSIZE_y), dpi=dpi)

    th_trans_upper_table = pd.read_csv(trans_band_upper_profs[i], sep='\s+', header=0)
    th_trans_upper_table = th_trans_upper_table.loc[:,'layer[0]':f'layer[{num_lay-1}]'].values

    th_trans_lower_table = pd.read_csv(trans_band_lower_profs[i], sep='\s+', header=0)
    th_trans_lower_table = th_trans_lower_table.loc[:,'layer[0]':f'layer[{num_lay-1}]'].values

    
    for nbin in range(35):
        ax_lo.plot(  th_trans_lower_table[nbin,:], init_table['P'], label='th')
        ax_up.plot(  th_trans_upper_table[nbin,:], init_table['P'], label='th')

    ax_up.set(xscale='linear', xlabel='transmission')
    ax_up.set(yscale='log', ylabel='pressure')
    ax_up.invert_yaxis()

    #ax_up.legend()
    
    ax_lo.set(xscale='linear', xlabel='transmission')
    ax_lo.set(yscale='log', ylabel='pressure')
    ax_lo.invert_yaxis()

    #ax_lo.legend()

# single scat albedo

In [ ]:
def E_i2s(w0, g0, i2s): 
    if g0 >= 0.0:
        r = np.ones(w0.shape)
        #print(w0[w0 > i2s])
        r[w0 > i2s] = np.maximum(1.0,  1.225 - 0.1582 * g0 - 0.1777 * w0[w0 > i2s] - 0.07465 * np.power(1.0 * g0, 2.0) + 0.2351 * w0[w0 > i2s] * g0
                    - 0.05582 * np.power(w0[w0 > i2s], 2.0))
        
        return r
    else:
        return 1.0

def E_noi2s(w0, g0, i2s): 
    
        return np.maximum(1.0,  1.225 - 0.1582 * g0 - 0.1777 * w0 - 0.07465 * np.power(1.0 * g0, 2.0) + 0.2351 * w0 * g0
                    - 0.05582 * np.power(w0, 2.0))

    
def E_cst(w0, g0, i2s): 
    
        return 1.0

        
def g_num(w0, g0, E, epsilon, epsilon2):
    return w0 * (E * (1.0 - w0 * g0) + g0 * epsilon / epsilon2)

def g_denom(w0, g0, E, mu, epsilon):
    return E*np.power(mu/epsilon, 2.0)*(E-w0)*(1-w0*g0) - 1.0

    # double num = w0 * (E * (1.0 - w0 * g0) + g0 * epsilon / epsilon2);

    # double denom = E * pow(mu_star / epsilon, 2.0) * (E - w0) * (1.0 - w0 * g0) - 1.0;
    
def g_second_plus(w0, g0, E, mu, epsilon):
    return mu / epsilon + 1.0 / (E * (1.0 - w0 * g0));

def g_second_minus(w0, g0, E, mu, epsilon):
    return mu / epsilon - 1.0 / (E * (1.0 - w0 * g0));


def g_third(w0, g0, E, epsilon, epsilon2):
    return epsilon * w0 * g0 / (epsilon2 * E * (1.0 - w0 * g0));

def G_plus(w0, g0, E, mu, epsilon, epsilon2):
    return 0.5*(g_num(w0, g0, E, epsilon, epsilon2) / g_denom(w0, g0, E, mu, epsilon) * g_second_plus(w0, g0, E, mu, epsilon) + g_third(w0, g0, E, epsilon, epsilon2))

def G_minus(w0, g0, E, mu, epsilon, epsilon2):
    return 0.5*(g_num(w0, g0, E, epsilon, epsilon2) / g_denom(w0, g0, E, mu, epsilon) * g_second_minus(w0, g0, E, mu, epsilon) - g_third(w0, g0, E, epsilon, epsilon2))

    
def G_denom_roots(g0, E, mu, epsilon):
    a = g0
    b  = 1.0+g0*E
    c = 1.0/(E*np.power(mu/epsilon, 2.0)) - E
    disc = np.sqrt(b*b-4.0*a*c)
    if g0 != 0.0:    
        return (-b + disc)/(2.0*a), (-b - disc)/(2.0*a)
    else:
        
        return (-c,-c)
    




g0 = 0.0
i2s = 0.1
epsilon = 0.5
epsilon_2 = 2.0/3.0
mu_star_l = mu_star*1.000

ver_coord_pressure = False

E_f = E_i2s

roots = G_denom_roots(g0, 1.0, mu_star_l, epsilon)

print(f"G_denom_roots: {roots}")

# root with the E function
root_result = root_scalar(lambda x: g_denom(np.array(x), g0, E_f(np.array(x), g0, i2s), mu_star_l, epsilon), bracket=(0.0,1.0), x0=roots[0], x1=roots[1]  )
root_E = root_result.root

if iso:
    


    fig, ((ax_omega, ax_omega_lin),(ax_fex, ax_g_denom), ( ax_1ofex, ax_numdenom ), (ax_g_plus, ax_g_minus)) = plt.subplots(4, 2, figsize=(2*FIGSIZE_x, 4*FIGSIZE_y), dpi=dpi)
    
    col_TP_table = pd.read_csv(init_profs[col_idx], sep='\s+', header=None, skiprows=2, names=['T','P'])
    if ver_coord_pressure:
        y_coord = col_TP_table['P']
    else:
        y_coord = np.arange(len(col_TP_table['P']))
    
    for nbin in range(num_bins):
        th_singscat_table = pd.read_csv(singscat_band_profs[col_idx], sep='\s+', header=0)
        th_w0_table = th_singscat_table.loc[:,'layer[0]':f'layer[{num_lay-1}]'].values
        w0_ = th_w0_table[nbin,:]
        th_num = g_num(w0_, g0, E_f(w0_, g0, i2s), epsilon, epsilon_2)
        th_fex = g_denom(w0_, g0, E_f(w0_, g0, i2s), mu_star_l, epsilon, )
        
        ax_omega.plot(  th_w0_table[nbin,:], y_coord, label=f'bin[{nbin}]')
        ax_omega_lin.plot(  th_w0_table[nbin,:], y_coord, label=f'bin[{nbin}]')
        ax_omega_lin.scatter(  th_w0_table[nbin,:], y_coord, label=f'bin[{nbin}]', marker='x')
        
        # mark the ones with negative denom
        ax_omega_lin.scatter(  th_w0_table[nbin,:][th_fex < 0.0] , y_coord[th_fex < 0.0], label=f'bin[{nbin}]', marker='+', color='r')
        
        ax_fex.scatter(th_fex,y_coord, label=f'bin[{nbin}]', marker='x')
        ax_fex.plot(th_fex, y_coord, label=f'bin[{nbin}]', marker='x')

        ax_1ofex.scatter(1.0/th_fex, y_coord, label=f'bin[{nbin}]', marker='x')
        ax_1ofex.plot(1.0/th_fex, y_coord, label=f'bin[{nbin}]', marker='x')
        
        ax_numdenom.scatter(th_num/th_fex, y_coord, label=f'bin[{nbin}]', marker='x')
        ax_numdenom.plot(th_num/th_fex, y_coord, label=f'bin[{nbin}]', marker='x')
        
        Gp =  G_plus(w0_, g0, E_f(w0_, g0, i2s), mu_star_l, epsilon, epsilon_2)
        ax_g_plus.scatter(Gp, y_coord, label=f'bin[{nbin}]', marker='x')
        ax_g_plus.plot(Gp, y_coord, label=f'bin[{nbin}]', marker='x')
        
        Gm =  G_minus(w0_, g0, E_f(w0_, g0, i2s), mu_star_l, epsilon, epsilon_2)
        ax_g_minus.scatter(Gm, y_coord, label=f'bin[{nbin}]', marker='x')
        ax_g_minus.plot(Gm, y_coord, label=f'bin[{nbin}]', marker='x')
        
        
    omega_int = np.linspace(0.0, 1.0, 100)
    ax_g_denom.plot(omega_int, g_denom(omega_int, g0, E_f(omega_int, g0, i2s), mu_star_l, epsilon, ), label="G_denom")
    ax_g_denom_G_pm = ax_g_denom.twinx()
    ax_g_denom_G_pm.plot(omega_int, G_plus(omega_int, g0, E_f(omega_int, g0, i2s), mu_star_l, epsilon, epsilon_2), c='g', label="G_plus")
    ax_g_denom_G_pm.plot(omega_int, G_minus(omega_int, g0, E_f(omega_int, g0, i2s), mu_star_l, epsilon, epsilon_2), c='orange', label="G_minus")
    ax_g_denom.grid(True)
    ax_g_denom.axhline(0.0)
    ax_g_denom.axvline(root_E)
    ax_g_denom.set(xlabel="$\omega_0$", ylabel="G_denom")
    ax_g_denom.legend()
        
    ax_omega.set(xscale='log', xlabel='single scattering albedo')
    if ver_coord_pressure:
        ax_omega.set(yscale='log', ylabel='pressure')
        ax_omega.invert_yaxis()
    else:
        ax_omega.set(yscale='linear', ylabel='level')
    ax_omega.axvline(i2s)
    ax_omega.axvline(root_E)
    ax_omega.grid(True)
    
    ax_omega_lin.set(xscale='linear', xlabel='single scattering albedo')
    if ver_coord_pressure:
        ax_omega_lin.set(yscale='log', ylabel='pressure')
        ax_omega_lin.invert_yaxis()
    else:
        ax_omega_lin.set(yscale='linear', ylabel='level')
    ax_omega_lin.axvline(i2s)
    ax_omega_lin.axvline(root_E)
    ax_omega_lin.grid(True)
    
    dx = 1.0e-2
    #ax_fex.set(xscale='linear', xlabel='$G_\pm$ denom', xlim=[-dx,dx])
    ax_fex.set(xscale='linear', xlabel='$G_\pm$ denom')
    if ver_coord_pressure:
        ax_fex.set(yscale='log', ylabel='pressure')
        ax_fex.invert_yaxis()
    else:
        ax_fex.set(yscale='linear', ylabel='level')
    ax_fex.axvline(0.0)
    ax_fex.grid(True)
    
    ax_1ofex.set(xscale='linear', xlabel=r'$1 / G_\pm$ denom')
    if ver_coord_pressure:
        ax_1ofex.set(yscale='log', ylabel='pressure')
        ax_1ofex.invert_yaxis()
    else:
        ax_1ofex.set(yscale='linear', ylabel='level')
    ax_1ofex.axvline(0.0)
    ax_1ofex.grid(True)
    #ax_1ofex.legend()
    
    ax_numdenom.set(xscale='linear', xlabel='$G_\pm$ num/denom')
    if ver_coord_pressure:
        ax_numdenom.set(yscale='log', ylabel='pressure')
        ax_numdenom.invert_yaxis()
    else:
        ax_numdenom.set(yscale='linear', ylabel='level')
    ax_numdenom.axvline(0.0)
    ax_numdenom.grid(True)
    
    
    ax_g_plus.set(xscale='linear', xlabel='$G_+$')
    if ver_coord_pressure:
        ax_g_plus.set(yscale='log', ylabel='pressure')
        ax_g_plus.invert_yaxis()
    else:
        ax_g_plus.set(yscale='linear', ylabel='level')
    ax_g_plus.axvline(0.0)
    ax_g_plus.grid(True)
    
    ax_g_minus.set(xscale='linear', xlabel='$G_-$')
    if ver_coord_pressure:
        ax_g_minus.set(yscale='log', ylabel='pressure')
        ax_g_minus.invert_yaxis()
    else:
        ax_g_minus.set(yscale='linear', ylabel='level')
    ax_g_minus.axvline(0.0)
    ax_g_minus.grid(True)

In [ ]:
if not iso:



    fig, ((ax1, ax2), (ax3,ax4)) = plt.subplots(2, 2, figsize=(2*FIGSIZE_x, 2*FIGSIZE_y), dpi=dpi)

    
    go = 0.0
    ms = -0.7
    epsi = 0.5

    for nbin in range(num_bins):
        th_singscat_upper_table = pd.read_csv(singscat_band_upper_profs[i], sep='\s+', header=0)
        th_w0_upper_table = th_singscat_upper_table.loc[:,'layer[0]':f'layer[{num_lay-1}]'].values

        ax1.plot(  th_w0_upper_table[nbin,:], init_table['P'], label=f'th[i]')
    
        th_singscat_lower_table = pd.read_csv(singscat_band_lower_profs[i], sep='\s+', header=0)
        th_w0_lower_table = th_singscat_lower_table.loc[:,'layer[0]':f'layer[{num_lay-1}]'].values

        ax2.plot(  th_w0_lower_table[nbin,:], init_table['P'], label=f'th[{nbin}]')
        
    
        wo = th_w0_lower_table[nbin,:]
        
        
        #L = ((1-wo)*(1-wo*go) - 1)/(pow(ms, -2) + 1/epsi*(1-wo)*(1-wo*go) )
        #g_p = 0.5*(L*(1/epsi + 1.0/(ms*1-wo*go)) + wo*go*ms/(1 - wo*go) )
        g_p = 0.5*((-wo/(1-pow(ms/epsi,2)*(1-wo)))*(pow(ms,2)/epsi+ms))
        
        
        g_m = 0.5*((-wo/(1-pow(ms/epsi,2)*(1-wo)))*(pow(ms,2)/epsi-ms))
    
    
        ax3.plot(  g_p, init_table['P'], label=f'th[i]')
        ax4.plot(  g_m, init_table['P'], label=f'th[i]')
        
    
    ax1.set(xscale='linear', xlabel='single scattering albedo')
    ax1.set(yscale='log', ylabel='pressure')
    ax1.invert_yaxis()
    ax1.set_title("$\omega_0$ upper")
    
    if go != 0:
        a = go
        d = 1
        c = pow(ms/epsi, 2)
    
        det = pow(-1 - a * d,2) - 4*a*(d-1/c)
        dp = (1 + a*d + math.sqrt(det))/(4*a*(d-1/c))
        dm = (1 + a*d - math.sqrt(det))/(4*a*(d-1/c))
        ax2.vlines(dp)
        ax2.vlines(dm)

    else:
        l = 1 - pow(epsi/ms, 2.0)
    
        ax2.axvline(l)
        print(f"lim w_0: {l}" )
    
    ax2.set(xscale='log', xlabel='single scattering albedo')
    #ax2.set(xlim=[1-0.1, 1], yscale='log', ylabel='pressure')
    ax2.set( yscale='log', ylabel='pressure')
    ax2.invert_yaxis()
    ax2.set_title("$\omega_0$ lower")

    
    ax3.set(xscale='linear', xlabel='$G_+$ calc')
    ax3.set(yscale='log', ylabel='pressure')
    ax3.invert_yaxis()
    ax3.set_title("G_plus_lower")
    
    
    ax4.set(xscale='linear', xlabel='$G_-$ calc')
    ax4.set(yscale='log', ylabel='pressure')
    ax4.invert_yaxis()
    ax4.set_title("G_minus_lower")
    
    #ax4.legend()


# Flux

### global

In [ ]:
nbin = num_bins

ver_unit_pressure = False

fig, ((ax_down_th, ax_down_hl),
     (ax_up_th, ax_up_hl),
     (ax_tot_th, ax_tot_hl)) = plt.subplots(3, 2, figsize=(2*FIGSIZE_x, 3*FIGSIZE_y), dpi=dpi)

n = col_idx
delta = 16
stride = 1

rng = list(range(n, n + delta, stride)) #+ [col_idx]

for i in rng:
    
    fu_table = pd.read_csv(Fu_profs[i], sep='\s+', header=0)
    fu_tot = ((fu_table.loc[:,'interface[0]':f'interface[{num_lay}]']).multiply(fu_table['deltalambda'], axis='index')).sum().values
    
    fd_table = pd.read_csv(Fd_profs[i], sep='\s+', header=0)
    fd_val = fd_table.loc[:,'interface[0]':f'interface[{num_lay}]'].values
    
    fd_tot = ((fd_table.loc[:,'interface[0]':f'interface[{num_lay}]']).multiply(fd_table['deltalambda'], axis='index')).sum().values
    
    fdir_table = pd.read_csv(Fdir_profs[i], sep='\s+', header=0)
    fdir_tot = ((fdir_table.loc[:,'interface[0]':f'interface[{num_lay}]']).multiply(fdir_table['deltalambda'], axis='index')).sum().values
    f_tot = fdir_tot + fd_tot 
    #f_tot = fd_tot 
    
    interface_table_col = pd.read_csv(interface_profs[i], sep='\s+', header=None, skiprows=2, names=['T','P','rho'])['P']
    if ver_unit_pressure:
        ver_coord = interface_table_col
    else:
        ver_coord = np.arange(len(interface_table_col))
    
     
    
    
    ax_down_th.plot(  f_tot, ver_coord, label=f'col[{i}]')
    ax_up_th.plot(  fu_tot, ver_coord, label=f'col[{i}]')
    ax_tot_th.plot(  fu_tot - f_tot, ver_coord, label=f'col[{i}]')

        
    
if has_hl:    
    hlintflxfile = hldmp / f"alfprof_disc_{col_idx}" /  f"alfprof_disc_{col_idx}_integrated_flux.dat" 

    hl_intflx =  pd.read_csv(hlintflxfile, sep='\s+', header=0, skiprows=2)

    hl_fdir = hl_intflx['F_dir']
    hl_fdown = hl_intflx['F_down']
    hl_press = hl_intflx['press.[10^-6bar]']*1e-6
    ax_down_hl.plot((hl_fdir + hl_fdown)*1e-3, hl_press)
    
    
if ver_unit_pressure: 
    vscale = 'log'
    vlabel = 'pressure [bar]'
else:
    vscale = 'linear'
    vlabel = 'level'
    
ax_down_th.set(xscale='linear', xlabel='Downward flux')
ax_down_th.set(yscale=vscale, ylabel=vlabel)

if ver_unit_pressure:
    ax_down_th.invert_yaxis()
ax_down_th.grid(True)
#ax_down_th.legend()

ax_up_th.set(xscale='linear', xlabel='Upward flux')
ax_up_th.set(yscale=vscale, ylabel=vlabel)

if ver_unit_pressure:
    ax_up_th.invert_yaxis()
ax_up_th.grid(True)

ax_tot_th.set(xscale='linear', xlabel='Net flux')
ax_tot_th.set(yscale=vscale, ylabel=vlabel)

if ver_unit_pressure:
    ax_tot_th.invert_yaxis()
ax_tot_th.grid(True)
# ax.legend()

### X_buff

In [ ]:
pd.read_csv(X_buff_profs[col_idx], sep='\s+', header=0, skiprows=0).loc[:,'int[0]':f'int[{2*(2*num_lay+1) -1}]']

In [ ]:
pd.read_csv(X_buff_profs[col_idx], sep='\s+', header=0, skiprows=0)

In [ ]:


ver_unit_pressure = False

weight_per_band = False

fig, ((ax_down, ax_up),(ax_down_tot, ax_up_tot), (ax_dir, ax_dummy)) = plt.subplots(3, 2, figsize=(2*FIGSIZE_x, 3*FIGSIZE_y), dpi=dpi)


fu_interface_tot = np.zeros(num_lay+1)
fd_interface_tot = np.zeros(num_lay+1)
fu_layer_tot = np.zeros(num_lay)
fd_layer_tot = np.zeros(num_lay)

fdir_layer_tot = np.zeros(num_lay)
fdir_interface_tot = np.zeros(num_lay+1)

c_cycler =  cycler(color=plt.rcParams['axes.prop_cycle'].by_key()['color'])()

for i in range(0,35):
    cl = next(c_cycler)['color']
    X_buff = pd.read_csv(X_buff_profs[col_idx], sep='\s+', header=0, skiprows=0).loc[:,'int[0]':f'int[{2*(2*num_lay+1)-1}]'].values
    
    fdir_interface = pd.read_csv(Fdir_profs[i], sep='\s+', header=0).loc[:,'interface[0]':f'interface[{num_lay}]'].values[i,:]
    fdir_layer = pd.read_csv(Fcdir_profs[i], sep='\s+', header=0).loc[:,'layer[0]':f'layer[{num_lay-1}]'].values[i,:]
    
    
    deltalambda = pd.read_csv(X_buff_profs[col_idx], sep='\s+', header=0, skiprows=0).loc[i,'deltalambda']
    
    # print(X_buff.shape)
    # idx: 
    # 0: lay up
    # 1: lay down
    # 2: int up
    # 3: int down
    fu_interface  = X_buff[i,0::4]
    fd_interface = X_buff[i,1::4]
    fu_layer   = X_buff[i,2::4]
    fd_layer   = X_buff[i,3::4]
    
    fu_interface_tot += fu_interface*deltalambda
    fd_interface_tot += fd_interface*deltalambda
    fu_layer_tot += fu_layer*deltalambda
    fd_layer_tot += fd_layer*deltalambda
    fdir_layer_tot += fdir_layer*deltalambda
    fdir_interface_tot += fdir_interface*deltalambda
    #print(fu_layer.shape,fd_layer.shape,fu_interface.shape,fd_interface.shape)
    
    if ver_unit_pressure:
        ver_layer_unit = layer_pressure
        ver_interface_unit = interface_pressure
    else:
        ver_layer_unit = np.arange(len(layer_pressure)) + 0.5
        ver_interface_unit = np.arange(len(interface_pressure)) 
        
    ax_down.plot(  fd_layer, ver_layer_unit, label=f'col[{i}]', ls='None', marker='+', lw=2, color=cl)
    ax_up.plot(  fu_layer, ver_layer_unit, label=f'col[{i}]', ls='None', marker='+', lw=2, color=cl)
    ax_down.plot(  fd_interface, ver_interface_unit, label=f'col[{i}]', ls='None', marker='x', lw=3, color=cl)
    ax_up.plot(  fu_interface, ver_interface_unit, label=f'col[{i}]', ls='None', marker='x', lw=3, color=cl)

    ax_dir.plot(  fdir_interface, ver_interface_unit, label=f'col[{i}]', ls='None', marker='x', lw=3, color=cl)
    ax_dir.plot(  fdir_layer, ver_layer_unit, label=f'col[{i}]', ls='None', marker='+', lw=2, color=cl)
    
if ver_unit_pressure: 
    vscale = 'log'
    vlabel = 'pressure [bar]'
else:
    vscale = 'linear'
    vlabel = 'level'
    
ax_down.set(xscale='linear', xlabel='Downward flux')
ax_down.set(yscale=vscale, ylabel=vlabel)

ax_down.set_ylim(30,41)

if ver_unit_pressure:
    ax_down.invert_yaxis()
ax_down.grid(True)
#ax_down_th.legend()

ax_up.set(xscale='linear', xlabel='Upward flux')
ax_up.set(yscale=vscale, ylabel=vlabel)

if ver_unit_pressure:
    ax_up.invert_yaxis()
ax_up.grid(True)



    
ax_down_tot.plot(  fd_layer_tot, ver_layer_unit, label=f'col[{i}]', ls='None', marker='+', lw=2, color=cl)
ax_up_tot.plot(  fu_layer_tot, ver_layer_unit, label=f'col[{i}]', ls='None', marker='+', lw=2, color=cl)
ax_down_tot.plot(  fd_interface_tot, ver_interface_unit, label=f'col[{i}]', ls='None', marker='x', lw=3, color=cl)
ax_up_tot.plot(  fu_interface_tot, ver_interface_unit, label=f'col[{i}]', ls='None', marker='x', lw=3, color=cl)

if ver_unit_pressure: 
    vscale = 'log'
    vlabel = 'pressure [bar]'
else:
    vscale = 'linear'
    vlabel = 'level'
    
ax_down_tot.set(xscale='linear', xlabel='Downward flux')
ax_down_tot.set(yscale=vscale, ylabel=vlabel)

if ver_unit_pressure:
    ax_down_tot.invert_yaxis()
ax_down_tot.grid(True)
#ax_down_th.legend()

ax_up_tot.set(xscale='linear', xlabel='Upward flux')
ax_up_tot.set(yscale=vscale, ylabel=vlabel)

if ver_unit_pressure:
    ax_dir.invert_yaxis()
ax_dir.grid(True)


ax_dir.set(xscale='linear', xlabel='Directional flux')
ax_dir.set(yscale=vscale, ylabel=vlabel)

if ver_unit_pressure:
    ax_dir.invert_yaxis()
ax_dir.grid(True)
#ax_down_th.legend()

### column inspection

In [ ]:
nbin = num_bins

fig, ax = plt.subplots(1, 1, figsize=(1*FIGSIZE_x, FIGSIZE_y), dpi=dpi)

n = col_idx
n = 320
delta = 1
for i in range(n, n + delta, 1):
    fu_table = pd.read_csv(Fu_profs[i], sep='\s+', header=0)
    fu_tot = ((fu_table.loc[:,'interface[0]':f'interface[{num_lay}]']).multiply(fu_table['deltalambda'], axis='index')).sum().values
    
    fd_table = pd.read_csv(Fd_profs[i], sep='\s+', header=0)
    fd_val = fd_table.loc[:,'interface[0]':f'interface[{num_lay-1}]'].values
    
    fd_tot = ((fd_table.loc[:,'interface[0]':f'interface[{num_lay}]']).multiply(fd_table['deltalambda'], axis='index')).sum().values
    
    fdir_table = pd.read_csv(Fdir_profs[i], sep='\s+', header=0)
    fdir_tot = ((fdir_table.loc[:,'interface[0]':f'interface[{num_lay}]']).multiply(fdir_table['deltalambda'], axis='index')).sum().values
    f_tot = fdir_tot + fd_tot 
    
    ax.plot(  f_tot, interface_table['P'], label=f'col[{i}]')

if has_hl:
    hl_fdir = hl_intflx['F_dir']
    hl_fdown = hl_intflx['F_down']
    hl_press = hl_intflx['press.[10^-6bar]']*1e-6
    ax.plot((hl_fdir + hl_fdown)*1e-3, hl_press)
    
ax.set(xscale='linear', xlabel='Downward flux')
ax.set(yscale='log', ylabel='pressure')
ax.invert_yaxis()
ax.grid(True)
# ax.legend()

### wavelength bins

In [ ]:
nbin = num_bins 
#nbin = 0

nbin_min = 0
#nbin_min = 10

fig, ((ax1, ax2), (ax3, ax4), (ax_fnet, ax_dummy1),(ax_fum, ax_fdm), (ax_ftm, ax_fdirm), (ax_fnetm, ax_dummy2)) = plt.subplots(6, 2, figsize=(2*FIGSIZE_x, 6*FIGSIZE_y), dpi=dpi)

i = col_idx

fu_table_col = pd.read_csv(Fu_profs[i], sep='\s+', header=0)
fd_table_col = pd.read_csv(Fd_profs[i], sep='\s+', header=0)
fdir_table_col = pd.read_csv(Fdir_profs[i], sep='\s+', header=0)


log_xscale = False
log_xsymscale = True
ver_unit_pressure = False
log_fnet = True

fu_map = np.zeros((len(interface_pressure), nbin ))
fd_map = np.zeros((len(interface_pressure), nbin ))
fdir_map = np.zeros((len(interface_pressure), nbin ))
ft_map = np.zeros((len(interface_pressure), nbin ))

if ver_unit_pressure:
    ver_interface_unit = interface_pressure
else:
    ver_interface_unit = np.arange(len(interface_pressure)) 

for n in range(nbin_min,nbin):
    
    fu_tot = fu_table_col.loc[n,'interface[0]':f'interface[{num_lay}]']
    
    fd_tot = fd_table_col.loc[n,'interface[0]':f'interface[{num_lay}]']
    
    fdir_tot = fdir_table_col.loc[n,'interface[0]':f'interface[{num_lay}]']
    f_tot = fdir_tot + fd_tot 
    f_net = fu_tot - f_tot
    
    fu_map[:,n] = fu_tot
    fd_map[:,n] = fd_tot
    fdir_map[:,n] = fdir_tot
    ft_map[:,n] = f_tot
    
    
    ax1.plot(  f_tot, ver_interface_unit, label=f'bin[{n}]')
    ax2.plot(  fdir_tot, ver_interface_unit, label=f'bin[{n}]')
    ax3.plot(  fd_tot, ver_interface_unit, label=f'bin[{n}]')
    ax4.plot(  fu_tot, ver_interface_unit, label=f'bin[{n}]')
    ax_fnet.plot(  f_net, ver_interface_unit, label=f'bin[{n}]')

    
fnet_map = fu_map - ft_map
    
if ver_unit_pressure: 
    vscale = 'log'
    vlabel = 'pressure [bar]'
else:
    vscale = 'linear'
    vlabel = 'level'

if log_xscale:
    xscale = 'log'
else:
    xscale = 'linear'

if log_xsymscale:
    xsymscale = 'symlog'
else:
    xsymscale = 'linear'

    
ax1.set(xscale=xscale, xlabel='Downward flux')
ax1.set(yscale=vscale, ylabel=vlabel)
#ax1.set_xlim(-0.1e11, 0.2e11)
#ax1.set_ylim(30, 41)
if ver_unit_pressure: 
    ax1.invert_yaxis()
ax1.grid(True)
#ax1.legend()
ax1.set_title("Downward Total Flux")

ax2.set(xscale=xscale, xlabel='Downward flux')
ax2.set(yscale=vscale, ylabel=vlabel)
#ax2.set_xlim(-0.1e11, 0.2e11)
#ax2.set_ylim(37, 41)
if ver_unit_pressure: 
    ax2.invert_yaxis()
ax2.grid(True)
#ax2.legend()
ax2.set_title("Downward Directional Flux")

ax3.set(xscale=xscale, xlabel='Downward flux')
ax3.set(yscale=vscale, ylabel=vlabel)
#ax3.set_xlim(-0.01e10, 0.1e10)
#ax3.set_ylim(30, 41)

if ver_unit_pressure: 
    ax3.invert_yaxis()
ax3.grid(True)
#ax3.legend()
ax3.set_title("Downward Diffuse Flux")

ax4.set(xscale=xscale, xlabel='Upward flux')
ax4.set(yscale=vscale, ylabel=vlabel)
#ax4.set_xlim(-0.01e12, .2e12)
#ax4.set_ylim(30, 41)

if ver_unit_pressure: 
    ax4.invert_yaxis()
ax4.grid(True)
#ax4.legend()
ax4.set_title("Up Diffuse Flux")

ax_fnet.set(xscale=xsymscale, xlabel='Net flux')
ax_fnet.set(yscale=vscale, ylabel=vlabel)
#ax_fnet.set_xlim(-0.1e11, .2e11)
#ax_fnet.set_ylim(30, 41)
if ver_unit_pressure: 
    ax_fnet.invert_yaxis()
ax_fnet.grid(True)
#ax1.legend()
ax_fnet.set_title("Net Flux")

pos = ax_fum.imshow(fu_map, origin='lower', norm=LogNorm())
ax_fum.set_title("Up Diffuse Flux")
ax_fum.set_xlabel('bin')
ax_fum.set_ylabel('interface')
fig.colorbar(pos, ax=ax_fum)

pos = ax_fdm.imshow(fd_map, origin='lower', norm=LogNorm())
ax_fdm.set_title("Downward Diffuse Flux")
ax_fdm.set_xlabel('bin')
ax_fdm.set_ylabel('interface')
fig.colorbar(pos, ax=ax_fdm)

pos = ax_fdirm.imshow(fdir_map, origin='lower', norm=LogNorm())
ax_fdirm.set_title("Downward Directional Flux")
ax_fdirm.set_xlabel('bin')
ax_fdirm.set_ylabel('interface')
fig.colorbar(pos, ax=ax_fdirm)

pos = ax_ftm.imshow(ft_map, origin='lower', norm=LogNorm())
ax_ftm.set_title("Downward Total Flux")
ax_ftm.set_xlabel('bin')
ax_ftm.set_ylabel('interface')
fig.colorbar(pos, ax=ax_ftm)


if log_fnet:
    fnet_map_log = np.zeros_like(fnet_map)
    fnet_map_log[fnet_map > 0.0] = np.log(fnet_map[fnet_map > 0.0])
    fnet_map_log[fnet_map < 0.0] = -np.log(-fnet_map[fnet_map < 0.0])
    vmax=np.max(np.abs(fnet_map_log))
    pos = ax_fnetm.imshow(fnet_map_log, origin='lower', cmap='RdBu_r', vmax=vmax, vmin=-vmax)
    ax_fnetm.set_title("Net Flux")
    ax_fnetm.set_xlabel('bin')
    ax_fnetm.set_ylabel('interface')
    fig.colorbar(pos, ax=ax_fnetm)
else:
    vmax=np.max(np.abs(fnet_map))
    pos = ax_fnetm.imshow(fnet_map, origin='lower', cmap='RdBu_r', vmax=vmax, vmin=-vmax)
    ax_fnetm.set_title("Net Flux")
    ax_fnetm.set_xlabel('bin')
    ax_fnetm.set_ylabel('interface')
    fig.colorbar(pos, ax=ax_fnetm)


ax_dummy1.set_visible(False)
ax_dummy2.set_visible(False)

In [ ]:
i = col_idx

fd_table = pd.read_csv(Fd_profs[i], sep='\s+', header=0)
fd_val = fd_table.loc[:,'interface[0]':f'interface[{num_lay-1}]'].values
    
f_tot = ((fd_table.loc[:,'interface[0]':f'interface[{num_lay}]']).multiply(fd_table['deltalambda'], axis='index')).sum().values


In [ ]:
nbin = num_bins

i = col_idx

ver_coord_pressure = False


init_table_col = pd.read_csv(init_profs[col_idx], sep='\s+', header=None, skiprows=2, names=['T','P','rho'])

def plot_stuff(stuff, ax, n):
    stuff_table = pd.read_csv(stuff, sep='\s+', header=0)
    stuff_tot = stuff_table.loc[n,'layer[0]':f'layer[{num_lay-1}]']
    if ver_coord_pressure:
        ax.plot(  stuff_tot, init_table_col['P'], label=f'bin[{n}]')
    else:
        ax.plot(  stuff_tot, np.arange(len(init_table_col['P'])), label=f'bin[{n}]')

def set_scale(ax, title, xlabel, xlog=False, ylog=True, xlim=None):
    if xlog:
        xscale = 'log'
    else:
        xscale = 'linear'
    if ylog:
        yscale = 'log'
    else:
        yscale = 'linear'
    if xlim is not None:
        ax.set_xlim(xlim)
    
    ax.set(xscale=xscale, xlabel=xlabel)
    if ver_coord_pressure:
        ax.set(yscale=yscale, ylabel='pressure')
        ax.invert_yaxis()
    else:
        ax.set(yscale='linear', ylabel='level')
    ax.grid(True)
    #ax.legend()
    ax.set_title(title)
    
if iso:
    fig, (ax_m, ax_n, ax_p, ax_g_minus, ax_g_plus)  = plt.subplots(5, 1, figsize=(1*FIGSIZE_x, 5*FIGSIZE_y), dpi=dpi)

    
    for n in range(0,num_bins):
        plot_stuff(M_profs[i], ax_m, n)
        plot_stuff(N_profs[i], ax_n, n)
        plot_stuff(P_profs[i], ax_p, n)
        plot_stuff(G_plus_profs[i], ax_g_plus, n)
        plot_stuff(G_minus_profs[i], ax_g_minus, n)
    
    set_scale(ax_m, "M", "M")
    set_scale(ax_n, "N", "N", xlog=True)
    set_scale(ax_p, "P", "P")
    #set_scale(ax_g_plus, "G_plus", "G_plus", xlog=False, xlim=[-1.2,1.2])
    #set_scale(ax_g_minus, "G_minus", "G_minus", xlog=False, xlim=[-1.2,1.2])
    
    set_scale(ax_g_plus, "G_plus", "G_plus", xlog=False)
    set_scale(ax_g_minus, "G_minus", "G_minus", xlog=False)

if not iso:
    fig, ((ax_m_upper, ax_m_lower),
      (ax_n_upper, ax_n_lower),
      (ax_p_upper, ax_p_lower),
      (ax_g_plus_upper, ax_g_minus_upper),
      (ax_g_plus_lower, ax_g_minus_lower))  = plt.subplots(5, 2, figsize=(2*FIGSIZE_x, 5*FIGSIZE_y), dpi=dpi)

    
    for n in range(0,num_bins):
        plot_stuff(M_upper_profs[i], ax_m_upper, n)
        plot_stuff(N_upper_profs[i], ax_n_upper, n)
        plot_stuff(P_upper_profs[i], ax_p_upper, n)
        plot_stuff(G_plus_upper_profs[i], ax_g_plus_upper, n)
        plot_stuff(G_minus_upper_profs[i], ax_g_minus_upper, n)
    
        plot_stuff(M_lower_profs[i], ax_m_lower, n)
        plot_stuff(N_lower_profs[i], ax_n_lower, n)
        plot_stuff(P_lower_profs[i], ax_p_lower, n)
        plot_stuff(G_plus_lower_profs[i], ax_g_plus_lower, n)
        plot_stuff(G_minus_lower_profs[i], ax_g_minus_lower, n)
    
    set_scale(ax_m_upper, "M_upper", "M_upper")
    set_scale(ax_m_lower, "M_lower", "M_lower")

    set_scale(ax_n_upper, "N_upper", "N_upper", xlog=True)
    set_scale(ax_n_lower, "N_lower", "N_lower", xlog=True)


    set_scale(ax_p_upper, "P_upper", "P_upper")
    set_scale(ax_p_lower, "P_lower", "P_lower")

    #set_scale(ax_g_plus_upper, "G_plus_upper", "G_plus_upper", xlog=False, xlim=[-1.2,1.2])
    #set_scale(ax_g_plus_lower, "G_plus_lower", "G_plus_lower", xlog=False, xlim=[-1.2,1.2])

    #set_scale(ax_g_minus_upper, "G_minus_upper", "G_minus_upper", xlog=False, xlim=[-1.2,1.2])
    #set_scale(ax_g_minus_lower, "G_minus_lower", "G_minus_lower", xlog=False, xlim=[-1.2,1.2])

    set_scale(ax_g_plus_upper, "G_plus_upper", "G_plus_upper", xlog=False)
    set_scale(ax_g_plus_lower, "G_plus_lower", "G_plus_lower", xlog=False)

    set_scale(ax_g_minus_upper, "G_minus_upper", "G_minus_upper", xlog=False)
    set_scale(ax_g_minus_lower, "G_minus_lower", "G_minus_lower", xlog=False)



### F_dir

# Qheat

In [ ]:
nbin = num_bins

ver_unit_pressure = False

fig, (ax_alf, ax_dgrt) = plt.subplots(1, 2, figsize=(2*FIGSIZE_x, 1*FIGSIZE_y), dpi=dpi)

n = col_idx

#n = 0
delta = 1
stride = 1

rng = list(range(n, n + delta, stride)) # + [col_idx]

for i in rng:
    
    alf_qh_table = pd.read_csv(alf_qheat_profs[i], sep='\s+', header=0)
    alf_qh = alf_qh_table['qheat'].values
    
    
    init_table_col = pd.read_csv(init_profs[i], sep='\s+', header=None, skiprows=2, names=['T','P','rho'])['P']
    if ver_unit_pressure:
        ver_coord = init_table_col
    else:
        ver_coord = np.arange(len(init_table_col))
    
     
    
    
    ax_alf.plot(  alf_qh, ver_coord, label=f'col[{i}]')
    
    #dgrt_qh_table = pd.read_csv(dgrt_qheat_profs[i], sep='\s+', header=0)
    #dgrt_qh = dgrt_qh_table['qheat'].values
    
    

    
     
    
    
    #ax_dgrt.plot(  dgrt_qh, ver_coord, label=f'col[{i}]')


        
    
if ver_unit_pressure: 
    vscale = 'log'
    vlabel = 'pressure [bar]'
else:
    vscale = 'linear'
    vlabel = 'level'
    
ax_alf.set(xscale='linear', xlabel='QHeat, Alf')
ax_alf.set(yscale=vscale, ylabel=vlabel)

ax_alf.set_ylim(30,40)

if ver_unit_pressure:
    ax_alf.invert_yaxis()
ax_alf.grid(True)

#ax_alf.legend()


#ax_dgrt.set(xscale='linear', xlabel='QHEAT, DGRT')
#ax_dgrt.set(yscale=vscale, ylabel=vlabel)
#
#if ver_unit_pressure:
#    ax_dgrt.invert_yaxis()
#ax_dgrt.grid(True)
#
#ax_dgrt.legend()